# Speech Seperation Experiments

In this file, we will run basic experiments for speech seperation frameworks. This will detemrine which framwork we will be using for our app: 

- [SpeechBrain's SepFormer](https://github.com/speechbrain/speechbrain)
- [FaceBook's Svoice](https://github.com/facebookresearch/svoice)

### SepFormer

#### Set up dependencies

In [2]:
%pip install torchaudio>=2.1.0,<2.9 torch>=2.1.0,<2.9
%pip install speechbrain
# !wget https://raw.githubusercontent.com/speechbrain/speechbrain/develop/requirements.txt
# !sed -i '/lint-requirements.txt/d' requirements.txt
# !pip install -r requirements.txt

/bin/bash: line 1: 2.9: No such file or directory


In [3]:
!pip uninstall -y transformers
!pip install "transformers<5.0"
!pip uninstall -y huggingface_hub
!pip install "huggingface_hub<1.0"


Found existing installation: transformers 4.57.6
Uninstalling transformers-4.57.6:
  Successfully uninstalled transformers-4.57.6
  Using cached transformers-4.57.6-py3-none-any.whl.metadata (43 kB)
Using cached transformers-4.57.6-py3-none-any.whl (12.0 MB)
Found existing installation: huggingface_hub 0.36.2
Uninstalling huggingface_hub-0.36.2:
  Successfully uninstalled huggingface_hub-0.36.2
  Using cached huggingface_hub-0.36.2-py3-none-any.whl.metadata (15 kB)
Using cached huggingface_hub-0.36.2-py3-none-any.whl (566 kB)


In [4]:
pip uninstall -y torch torchaudio

Found existing installation: torch 2.8.0
Uninstalling torch-2.8.0:
  Successfully uninstalled torch-2.8.0
Found existing installation: torchaudio 2.8.0
Uninstalling torchaudio-2.8.0:
  Successfully uninstalled torchaudio-2.8.0


In [5]:
! pip install "torch>=2.1.0,<2.9" "torchaudio>=2.1.0,<2.9"
!pip install --upgrade speechbrain

  Using cached torch-2.8.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached torchaudio-2.8.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (7.2 kB)
Using cached torch-2.8.0-cp312-cp312-manylinux_2_28_x86_64.whl (887.9 MB)
Using cached torchaudio-2.8.0-cp312-cp312-manylinux_2_28_x86_64.whl (4.0 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.24.0+cpu requires torch==2.9.0, but you have torch 2.8.0 which is incompatible.


In [32]:
!pip install --upgrade speechbrain

In [7]:
import speechbrain

In [9]:
# check package versions for debugging purposes
import torch
import torchaudio
import speechbrain
import transformers
import huggingface_hub

print("torch:", torch.__version__)
print("torchaudio:", torchaudio.__version__)
print("speechbrain:", speechbrain.__version__)
print("transformers:", transformers.__version__)
print("huggingface_hub:", huggingface_hub.__version__)

torch: 2.8.0+cu128
torchaudio: 2.8.0+cu128
speechbrain: 1.0.3
transformers: 4.57.6
huggingface_hub: 0.36.2


In [10]:
# Complete patch for torchaudio 2.9.x compatibility
import torchaudio
import soundfile as sf
import torch

# Patch 1: Add missing list_audio_backends function
if not hasattr(torchaudio, "list_audio_backends"):
    torchaudio.list_audio_backends = lambda: ["soundfile"]

# Patch 2: Monkey patch the load function
original_load = torchaudio.load if not hasattr(torchaudio, '_original_load') else torchaudio._original_load

def patched_load(filepath, *args, **kwargs):
    data, samplerate = sf.read(filepath, dtype='float32')
    data = torch.from_numpy(data)
    
    if data.dim() == 1:
        data = data.unsqueeze(0)  # (samples,) -> (1, samples)
    else:
        data = data.T  # (samples, channels) -> (channels, samples)
    
    return data, samplerate

torchaudio._original_load = original_load
torchaudio.load = patched_load

# Patch 3: Monkey patch the save function
original_save = torchaudio.save if not hasattr(torchaudio, '_original_save') else torchaudio._original_save

def patched_save(filepath, src, sample_rate, channels_first=True, **kwargs):
    # src is expected to be (channels, samples) if channels_first=True
    # soundfile expects (samples, channels)
    
    if channels_first:
        if src.dim() == 1:
            data = src.unsqueeze(1)  # (samples,) -> (samples, 1)
        else:
            data = src.T  # (channels, samples) -> (samples, channels)
    else:
        data = src
    
    # Convert to numpy and save
    data_np = data.detach().cpu().numpy()
    sf.write(filepath, data_np, sample_rate)

torchaudio._original_save = original_save
torchaudio.save = patched_save

In [11]:
from speechbrain.inference.separation import SepformerSeparation as seporator 

model = seporator.from_hparams(
    source="speechbrain/sepformer-wsj02mix", 
    savedir='pretrained_models/sepformer-wsj02mix'
)

INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Using symlink found at '/content/pretrained_models/sepformer-wsj02mix/hyperparams.yaml'
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/sepformer-wsj02mix' if not cached
/usr/local/lib/python3.12/dist-packages/speechbrain/utils/torch_audio_backend.py:57: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  available_backends = torchaudio.list_audio_backends()
DEBUG:speechbrain.utils.parameter_transfer:Collecting files (or symlinks) for pretraining in pretrained_models/sepformer-wsj02mix.
INFO:speechbrain.utils.fetching:Fetch maskn

In [12]:
est_sources = model.separate_file(path='speechbrain/sepformer-wsj02mix/test_mixture.wav') 

INFO:speechbrain.utils.fetching:Fetch test_mixture.wav: Fetching from HuggingFace Hub 'speechbrain/sepformer-wsj02mix' if not cached


In [13]:
torchaudio.save("source1hat.wav", est_sources[:, :, 0].detach().cpu(), 8000)
torchaudio.save("source2hat.wav", est_sources[:, :, 1].detach().cpu(), 8000)

In [14]:
from IPython.display import Audio

Audio("source1hat.wav")


In [15]:
Audio("source2hat.wav")

### SVoice